In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count
spark = (
    SparkSession
    .Builder()
    .appName('cf-project')
    .getOrCreate()
)

In [2]:
ratings = spark.read.csv(path='./final_ratings.csv', header=True, inferSchema=True)
users = spark.read.csv(path='./dataset/Users.csv', header=True, inferSchema=True)
books =  spark.read.csv(path='./dataset/Books.csv', header=True, inferSchema=True)

In [3]:
ratings.show()

+------+----------+------+--------+-----------------+---------+------------+
|UserID|      ISBN|Rating|numBooks|num_books_to_mask|book_rank|integer_isbn|
+------+----------+------+--------+-----------------+---------+------------+
|   243|155874262X|     5|      10|                3|        1|         206|
|   243|0449006522|     6|      10|                3|        2|         481|
|   243|044023722X|     7|      10|                3|        3|          26|
|   243|0425163407|     9|      10|                3|        4|         180|
|   243|0385720106|     7|      10|                3|        5|          91|
|   243|0316899984|     7|      10|                3|        6|         172|
|   243|0316776963|     9|      10|                3|        7|          64|
|   243|0316601950|     9|      10|                3|        8|          12|
|   243|0060977493|     7|      10|                3|        9|         135|
|   243|0060915544|    10|      10|                3|       10|          98|

In [4]:
significant_users = (
    ratings
    .select('UserID')
    .distinct()
    .orderBy(col('UserID').asc())
)


significant_books = (
    ratings
    .select('ISBN', 'integer_isbn')
     .distinct()
    .orderBy(col('integer_isbn').asc())
)

print('signific users: ', significant_users.count())
print('signific book: ', significant_books.count())


signific users:  784
signific book:  537


In [5]:
significant_books.show()

+----------+------------+
|      ISBN|integer_isbn|
+----------+------------+
|0316666343|           0|
|0385504209|           1|
|0312195516|           2|
|0679781587|           3|
|0142001740|           4|
|0060928336|           5|
|0446672211|           6|
|059035342X|           7|
|0743418174|           8|
|0452282152|           9|
|043935806X|          10|
|044021145X|          11|
|0316601950|          12|
|0446310786|          13|
|067976402X|          14|
|0345370775|          15|
|0375727345|          16|
|0345337662|          17|
|0671027360|          18|
|0060930535|          19|
+----------+------------+
only showing top 20 rows



In [6]:
#user joiture:

users = (
    significant_users
    .join(users, on='UserID', how='left')
    .orderBy(col('UserID').asc())
)

print('users: ', users.count())
users.show()

users:  784
+------+--------------------+----+
|UserID|            Location| Age|
+------+--------------------+----+
|   243|arden hills, minn...|NULL|
|   254|minneapolis, minn...|24.0|
|   638|san diego, califo...|20.0|
|  1435|st. louis, missou...|36.0|
|  2766|  frisco, texas, usa|42.0|
|  4017|new orleans, loui...|48.0|
|  6242|calgary, alberta,...|NULL|
|  6251|wahiawa, hawaii, usa|32.0|
|  6543|strafford, missou...|34.0|
|  6563|burien, washingto...|31.0|
|  6575|seattle, washingt...|NULL|
|  7125|        apo, ae, usa|28.0|
|  7158|omaha, nebraska, usa|30.0|
|  7283|saint marys, geor...|30.0|
|  7346|sunnyvale, califo...|49.0|
|  7915|st. paul, minneso...|NULL|
|  8066|chicago, illinois...|NULL|
|  8067|owings mills, mar...|26.0|
|  8245|torrance, califor...|31.0|
|  8253|tulsa, oklahoma, usa|26.0|
+------+--------------------+----+
only showing top 20 rows



In [7]:
from pyspark.sql.functions import split

users = (
    users.withColumn('splited_location', split('Location', ','))
    .withColumn('City', col('splited_location')[1])
    .withColumn('Country', col('splited_location')[2])
    .select('UserID','City', 'Country')
)
users.show()

+------+-----------+-------+
|UserID|       City|Country|
+------+-----------+-------+
|   243|  minnesota|    usa|
|   254|  minnesota|    usa|
|   638| california|    usa|
|  1435|   missouri|    usa|
|  2766|      texas|    usa|
|  4017|  louisiana|    usa|
|  6242|    alberta| canada|
|  6251|     hawaii|    usa|
|  6543|   missouri|    usa|
|  6563| washington|    usa|
|  6575| washington|    usa|
|  7125|         ae|    usa|
|  7158|   nebraska|    usa|
|  7283|    georgia|    usa|
|  7346| california|    usa|
|  7915|  minnesota|    usa|
|  8066|   illinois|    usa|
|  8067|   maryland|    usa|
|  8245| california|    usa|
|  8253|   oklahoma|    usa|
+------+-----------+-------+
only showing top 20 rows



In [8]:
users.toPandas().to_csv('./users.csv', index=False)

In [9]:
books = (
    significant_books
    .join(books, on='ISBN', how='left')
    .orderBy(col('integer_isbn').asc())
)

books.show(10)

+----------+------------+--------------------+---------------+-----------------+--------------------+--------------------+--------------------+--------------------+
|      ISBN|integer_isbn|               Title|         Author|YearOfPublication|           Publisher|           ImageURLS|           ImageURLM|           ImageURLL|
+----------+------------+--------------------+---------------+-----------------+--------------------+--------------------+--------------------+--------------------+
|0316666343|           0|The Lovely Bones:...|   Alice Sebold|             2002|       Little, Brown|http://images.ama...|http://images.ama...|http://images.ama...|
|0385504209|           1|   The Da Vinci Code|      Dan Brown|             2003|           Doubleday|http://images.ama...|http://images.ama...|http://images.ama...|
|0312195516|           2|The Red Tent (Bes...|  Anita Diamant|             1998|         Picador USA|http://images.ama...|http://images.ama...|http://images.ama...|
|067978158

In [11]:
books = books.na.drop()

books.show()

+----------+------------+--------------------+------------------+-----------------+--------------------+--------------------+--------------------+--------------------+
|      ISBN|integer_isbn|               Title|            Author|YearOfPublication|           Publisher|           ImageURLS|           ImageURLM|           ImageURLL|
+----------+------------+--------------------+------------------+-----------------+--------------------+--------------------+--------------------+--------------------+
|0316666343|           0|The Lovely Bones:...|      Alice Sebold|             2002|       Little, Brown|http://images.ama...|http://images.ama...|http://images.ama...|
|0385504209|           1|   The Da Vinci Code|         Dan Brown|             2003|           Doubleday|http://images.ama...|http://images.ama...|http://images.ama...|
|0312195516|           2|The Red Tent (Bes...|     Anita Diamant|             1998|         Picador USA|http://images.ama...|http://images.ama...|http://images.

In [12]:
from pyspark.sql.functions import lower

books = (
    books
    .withColumn('Title', lower('Title'))
    .withColumn('Author', lower('Author'))
    .withColumn('Publisher', lower('Publisher'))
)

books.show()

+----------+------------+--------------------+------------------+-----------------+--------------------+--------------------+--------------------+--------------------+
|      ISBN|integer_isbn|               Title|            Author|YearOfPublication|           Publisher|           ImageURLS|           ImageURLM|           ImageURLL|
+----------+------------+--------------------+------------------+-----------------+--------------------+--------------------+--------------------+--------------------+
|0316666343|           0|the lovely bones:...|      alice sebold|             2002|       little, brown|http://images.ama...|http://images.ama...|http://images.ama...|
|0385504209|           1|   the da vinci code|         dan brown|             2003|           doubleday|http://images.ama...|http://images.ama...|http://images.ama...|
|0312195516|           2|the red tent (bes...|     anita diamant|             1998|         picador usa|http://images.ama...|http://images.ama...|http://images.

In [13]:
books.toPandas().to_csv('./books.csv', index=False)

In [14]:
(
    ratings
    .select('UserID', 'integer_isbn', 'Rating')
    .toPandas()
    .to_csv('./rating_by_user.csv', index=False)
)


In [15]:
(
    ratings
    .select('integer_isbn', 'UserID', 'Rating')
    .toPandas()
    .to_csv('./rating_by_book.csv', index=False)
)

In [16]:
print('books: ', books.count())
print('users: ', users.count())
print('rating: ', ratings.count())

books:  531
users:  784
rating:  14060
